In [1]:
# インタビュー評価システム
# Ollama経由でqwen3.0を使って面接内容を評価します

import os
import glob
import requests
import json
import re
from typing import List, Dict

print("インタビュー評価システムを開始します...")


インタビュー評価システムを開始します...


In [12]:
# Ollama APIクライアント関数
def query_ollama(prompt: str, model: str = "qwen3:4b") -> str:
    """
    Ollama経由でqwen3.0に質問を送信し、回答を取得します
    """
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.2,
        }
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()["response"]
    except Exception as e:
        return f"エラー: {e}"

In [13]:
interview_files = glob.glob("interview_history_*.txt")
print(f"見つかった面接履歴ファイル: {interview_files}")

見つかった面接履歴ファイル: ['interview_history_1749304656_887dacfd.txt']


In [14]:
for file_path in interview_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        print(content)

=== 面接履歴 ===

👨‍💼 面接官: マイクロサービスの欠点は何ですか？
------------------------------
🤖 候補者: マイクロサービスアーキテクチャには、確かにいくつかの重要な欠点があります。私の経験を踏まえて、主な課題を説明します：

1. 複雑性の増大
マイクロサービスは、システム全体の運用複雑性を大幅に増加させます。以前、大規模な金融システムの再設計プロジェクトで、20以上のマイクロサービスを管理した経験から、以下の複雑性の課題を実感しました：
- サービス間の通信管理
- 分散トランザクションの調整
- 障害発生時のトレースとデバッグの難しさ

2. パフォーマンスのオーバーヘッド
サービス間の通信は、ネットワークホップによって追加のレイテンシを生み出します。特に、
- REST APIやgRPCによるサービス間通信
- シリアライゼーションとデシリアライゼーションのオーバーヘッド
- ネットワーク遅延の増加

3. データ整合性の課題
分散システムでは、データの一貫性を保つことが非常に難しくなります：
- 分散トランザクションの実装の複雑さ
- イベント駆動型アーキテクチャでの eventual consistency の管理
- サガパターンなどの複雑な補償トランザクションの実装

4. テストの複雑化
マイクロサービスのテストは従来のモノリシックアプリケーションと比較して格段に難しくなります：
- 統合テストの複雑性
- モックとスタブの大量作成
- エンドツーエンドのテストシナリオの設計

5. インフラストラクチャの複雑さ
運用面でも大きな課題があります：
- Kubernetesなどのコンテナオーケストレーションの複雑な設定
- 継続的デプロイメントの管理
- モニタリングとロギングの複雑化

6. チームの分散と認知的負荷
組織的な観点からも課題があります：
- サービス間の明確な境界の維持
- チーム間のコミュニケーションオーバーヘッド
- 各サービスの技術的負債の管理

7. コストの増加
インフラとメンテナンスのコストが増大します：
- より多くのサーバーリソース
- 複雑なモニタリングツール
- 高度なスキルを持つエンジニアの必要性

これらの欠点は、マイクロサービスが万能ではないことを示していま

In [15]:
# https://cybozu.co.jp/recruit/entry/career/web-engineer-kintone-ai.html
requirements = """
    # 業務内容	
    生成AI技術を活用したプロダクトの機能開発を担当していただきます。
    技術領域はフロントエンド、バックエンド、インフラ、生成AI技術まで幅広く携われる機会があります。
    プロジェクトマネージャー、プロダクトマネージャー、プロダクトデザイナー、QAエンジニア、プロダクトライターとも密に連携し、チームワークを活かして効率よく品質の高いものづくりを実現しています。

    # 開発環境	
    フロントエンド
    - 言語：TypeScript, JavaScript
    - フレームワーク、ライブラリ：React, zustand, styled-components
    サーバーサイド
    - 言語：Python, Java
    - フレームワーク：FastAPI, Spring Framework
    - データベース：MySQL
    - 検索エンジン：Elasticsearch, Amazon OpenSearch Service
    - AWSサービス : Amazon Bedrock, AWS Lambda, Amazon ElastiCache, Amazon S3, etc
    その他
    -テスト：pytest, Jest, Mocha, JUnit, Selenium（Java）
    - CI：GitHub Actions
    - IaC : AWS CDK

    # 必須スキル・条件	
    - 「チームワークあふれる社会を創る」という企業理念への共感
    - Webアプリケーションの開発経験5年以上
    - コンピュータサイエンスの基礎知識（アルゴリズムとデータ構造、セキュリティ、ネットワークなど）
    - 生成AI技術の経験・強い関心

    # 歓迎スキル・条件	
    - 大量アクセスや大量データによる負荷を考慮したマルチテナントのWebアプリケーション開発経験
    - RAGの開発経験
    - LLMOpsの経験
    - fine-tuningやmodel distillation等を活用した開発経験
    - チームやプロジェクトをリードした経験
    
    # 求める人物像	
    - 公明正大な方
    - 多様な個性を重視する方
    - 自主自律、対話と議論を大切にする方
    """


In [16]:
prompt = "これからシニアソフトウェアエンジニアの募集要項、および面接官と候補者との面接の議事録を渡します。これらの内容を見て、募集要項を満たした質問が出来ているか、面接官の質問内容について5段階評価してください。改善できるところがあれば、その内容を示してください。\n 募集要項:" + requirements + "\n 質問と回答:" + content
text_answer = query_ollama(prompt)
print(text_answer)

<think>
Okay, let's tackle this evaluation. The user provided a recruitment specification and a interview transcript. They want me to evaluate whether the questions asked by the interviewer meet the recruitment criteria, and if there are areas for improvement.

First, I need to understand the recruitment requirements thoroughly. The job is for a senior software engineer, focusing on generating AI technologies. The role involves working on products using front-end, back-end, infrastructure, and generative AI. They collaborate with various team members, and the environment includes specific tech stacks like TypeScript, React, Python, FastAPI, etc. The required skills include 5+ years of web app development, computer science fundamentals, and experience with generative AI. They also welcome skills in RAG, LLMOps, model distillation, and leadership.

Now, looking at the interview questions and answers. The first question was about the drawbacks of microservices. The candidate discussed sev